<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
# SpaceX Folium Interactive Visual Analytics - Complete Implementation

import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from js import fetch
import io
from math import sin, cos, sqrt, atan2, radians

# =============================================================================
# データの読み込み
# =============================================================================

# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
print("SpaceX DataFrame Info:")
print(f"Total records: {len(spacex_df)}")
print(f"Columns: {spacex_df.columns.tolist()}")
print("\nFirst 5 rows:")
print(spacex_df.head())

# Get unique launch sites
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
print(f"\nUnique Launch Sites ({len(launch_sites_df)}):")
print(launch_sites_df)

SpaceX DataFrame Info:
Total records: 56
Columns: ['Launch Site', 'Lat', 'Long', 'class']

First 5 rows:
   Launch Site        Lat       Long  class
0  CCAFS LC-40  28.562302 -80.577356      0
1  CCAFS LC-40  28.562302 -80.577356      0
2  CCAFS LC-40  28.562302 -80.577356      0
3  CCAFS LC-40  28.562302 -80.577356      0
4  CCAFS LC-40  28.562302 -80.577356      0

Unique Launch Sites (4):
    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
print("\n=== TASK 1: Creating map with all launch sites ===")

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map with appropriate zoom to show all sites
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a circle and marker at NASA Johnson Space Center
circle = folium.Circle(
    nasa_coordinate, 
    radius=1000, 
    color='#d35400', 
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

# Add all launch sites to the map
for index, site in launch_sites_df.iterrows():
    coordinate = [site['Lat'], site['Long']]
    
    # Create a circle for each launch site
    circle = folium.Circle(
        coordinate, 
        radius=1000, 
        color='blue', 
        fill=True,
        fillColor='blue',
        fillOpacity=0.6
    ).add_child(folium.Popup(site['Launch Site']))
    
    # Create a marker with site name
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

print("TASK 1 Complete: All launch sites marked on map")



=== TASK 1: Creating map with all launch sites ===
TASK 1 Complete: All launch sites marked on map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [13]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [14]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [15]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [7]:
print("\n=== TASK 2: Adding success/failure markers ===")

# Display last 10 records for verification
print("Last 10 records of spacex_df:")
print(spacex_df.tail(10))

# Create marker cluster
marker_cluster = MarkerCluster()

# Function to assign marker color based on class
def assign_marker_color(launch_class):
    """
    Assign marker color based on launch class
    class=1: Success (green)
    class=0: Failure (red)
    """
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Apply function to create marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

print(f"\nMarker color distribution:")
print(spacex_df['marker_color'].value_counts())

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Create markers for each launch record
for index, record in spacex_df.iterrows():
    # Create coordinate list [lat, lon]
    coordinate = [record['Lat'], record['Long']]
    
    # Create marker with appropriate color
    marker = folium.Marker(
        coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"""
        <b>Launch Site:</b> {record['Launch Site']}<br>
        <b>Class:</b> {record['class']}<br>
        <b>Outcome:</b> {'Success' if record['class'] == 1 else 'Failure'}
        """
    )
    
    # Add marker to cluster
    marker_cluster.add_child(marker)

print("TASK 2 Complete: Success/failure markers added to map")




=== TASK 2: Adding success/failure markers ===
Last 10 records of spacex_df:
     Launch Site        Lat       Long  class
46    KSC LC-39A  28.573255 -80.646895      1
47    KSC LC-39A  28.573255 -80.646895      1
48    KSC LC-39A  28.573255 -80.646895      1
49  CCAFS SLC-40  28.563197 -80.576820      1
50  CCAFS SLC-40  28.563197 -80.576820      1
51  CCAFS SLC-40  28.563197 -80.576820      0
52  CCAFS SLC-40  28.563197 -80.576820      0
53  CCAFS SLC-40  28.563197 -80.576820      0
54  CCAFS SLC-40  28.563197 -80.576820      1
55  CCAFS SLC-40  28.563197 -80.576820      0

Marker color distribution:
marker_color
red      32
green    24
Name: count, dtype: int64
TASK 2 Complete: Success/failure markers added to map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [10]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [11]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [19]:
print("\n=== TASK 3: Adding distance analysis ===")

# Add Mouse Position to get coordinates
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    Returns distance in kilometers
    """
    # Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Example analysis for KSC LC-39A (Kennedy Space Center)
ksc_coordinate = [28.57333, -80.64664]  # KSC LC-39A coordinates
launch_site_lat, launch_site_lon = ksc_coordinate

# 1. Closest coastline point analysis
print("\n--- Coastline Distance Analysis ---")
coastline_lat, coastline_lon = 28.56367, -80.57163  # Example coastline coordinate
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create coastline marker
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline),
    ),
    popup=f"Coastline<br>Distance to KSC: {distance_coastline:.2f} km"
)
site_map.add_child(coastline_marker)

# Create line between launch site and coastline
coastline_line = folium.PolyLine(
    locations=[ksc_coordinate, [coastline_lat, coastline_lon]], 
    weight=2,
    color='red',
    opacity=0.7
)
site_map.add_child(coastline_line)

print(f"Distance to coastline: {distance_coastline:.2f} km")

# 2. Highway/Road distance analysis
print("\n--- Highway Distance Analysis ---")
highway_lat, highway_lon = 28.52058, -80.68094  # Example highway coordinate (US-1)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create highway marker
highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#e74c3c;"><b>%s</b></div>' % "{:.2f} KM".format(distance_highway),
    ),
    popup=f"Highway (US-1)<br>Distance to KSC: {distance_highway:.2f} km"
)
site_map.add_child(highway_marker)

# Create line between launch site and highway
highway_line = folium.PolyLine(
    locations=[ksc_coordinate, [highway_lat, highway_lon]], 
    weight=2,
    color='blue',
    opacity=0.7
)
site_map.add_child(highway_line)

print(f"Distance to highway: {distance_highway:.2f} km")

# 3. City distance analysis (Cocoa Beach)
print("\n--- City Distance Analysis ---")
city_lat, city_lon = 28.38598, -80.60905  # Cocoa Beach coordinates
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create city marker
city_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#9b59b6;"><b>%s</b></div>' % "{:.2f} KM".format(distance_city),
    ),
    popup=f"Cocoa Beach<br>Distance to KSC: {distance_city:.2f} km"
)
site_map.add_child(city_marker)

# Create line between launch site and city
city_line = folium.PolyLine(
    locations=[ksc_coordinate, [city_lat, city_lon]], 
    weight=2,
    color='purple',
    opacity=0.7
)
site_map.add_child(city_line)

print(f"Distance to city (Cocoa Beach): {distance_city:.2f} km")

# 4. Railway distance analysis
print("\n--- Railway Distance Analysis ---")
railway_lat, railway_lon = 28.51066, -80.68175  # Example railway coordinate
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create railway marker
railway_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#f39c12;"><b>%s</b></div>' % "{:.2f} KM".format(distance_railway),
    ),
    popup=f"Railway<br>Distance to KSC: {distance_railway:.2f} km"
)
site_map.add_child(railway_marker)

# Create line between launch site and railway
railway_line = folium.PolyLine(
    locations=[ksc_coordinate, [railway_lat, railway_lon]], 
    weight=2,
    color='orange',
    opacity=0.7
)
site_map.add_child(railway_line)

print(f"Distance to railway: {distance_railway:.2f} km")

# =============================================================================
# Additional Analysis for Other Launch Sites
# =============================================================================

print("\n=== Additional Analysis for All Launch Sites ===")

# Analyze proximities for all launch sites
proximity_analysis = []

for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_lat, site_lon = site['Lat'], site['Long']
    
    # Calculate distances to various proximities (using KSC coastline as reference)
    coast_dist = calculate_distance(site_lat, site_lon, coastline_lat, coastline_lon)
    
    # Add site-specific analysis marker
    analysis_marker = folium.Marker(
        [site_lat, site_lon],
        icon=folium.Icon(color='darkgreen', icon='info-sign'),
        popup=f"""
        <b>{site_name}</b><br>
        <b>Coordinates:</b> {site_lat:.4f}, {site_lon:.4f}<br>
        <b>Distance to reference coastline:</b> {coast_dist:.2f} km
        """
    )
    site_map.add_child(analysis_marker)
    
    proximity_analysis.append({
        'Launch Site': site_name,
        'Latitude': site_lat,
        'Longitude': site_lon,
        'Distance to Coastline (km)': round(coast_dist, 2)
    })

# Display proximity analysis results
proximity_df = pd.DataFrame(proximity_analysis)
print("\nProximity Analysis Summary:")
print(proximity_df)

# =============================================================================
# Map Legend and Final Display
# =============================================================================

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<p><b>Legend</b></p>
<p><i class="fa fa-circle" style="color:green"></i> Successful Launch</p>
<p><i class="fa fa-circle" style="color:red"></i> Failed Launch</p>
<p><i class="fa fa-circle" style="color:blue"></i> Launch Sites</p>
<p><i class="fa fa-circle" style="color:orange"></i> NASA JSC</p>
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

print("\n=== Analysis Complete ===")
print("TASK 3 Complete: Distance analysis and proximity mapping finished")




=== TASK 3: Adding distance analysis ===

--- Coastline Distance Analysis ---
Distance to coastline: 7.41 km

--- Highway Distance Analysis ---
Distance to highway: 6.76 km

--- City Distance Analysis ---
Distance to city (Cocoa Beach): 21.16 km

--- Railway Distance Analysis ---
Distance to railway: 7.77 km

=== Additional Analysis for All Launch Sites ===

Proximity Analysis Summary:
    Launch Site   Latitude   Longitude  Distance to Coastline (km)
0   CCAFS LC-40  28.562302  -80.577356                        0.58
1  CCAFS SLC-40  28.563197  -80.576820                        0.51
2    KSC LC-39A  28.573255  -80.646895                        7.43
3   VAFB SLC-4E  34.632834 -120.610745                     3827.51

=== Analysis Complete ===
TASK 3 Complete: Distance analysis and proximity mapping finished


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

<class 'NameError'>: name 'lines' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [22]:
# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<p><b>Legend</b></p>
<p><i class="fa fa-circle" style="color:green"></i> Successful Launch</p>
<p><i class="fa fa-circle" style="color:red"></i> Failed Launch</p>
<p><i class="fa fa-circle" style="color:blue"></i> Launch Sites</p>
<p><i class="fa fa-circle" style="color:orange"></i> NASA JSC</p>
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

print("\n=== Analysis Complete ===")
print("TASK 3 Complete: Distance analysis and proximity mapping finished")

# =============================================================================
# Geographic Insights Summary
# =============================================================================

print("\n=== GEOGRAPHIC INSIGHTS SUMMARY ===")
print(f"""
Key Findings:
1. Launch sites are strategically positioned near coastlines for safety and trajectory optimization
2. KSC LC-39A distances:
   - Coastline: {distance_coastline:.2f} km (very close for safety)
   - Highway: {distance_highway:.2f} km (accessible for logistics)
   - City: {distance_city:.2f} km (safe distance from population)
   - Railway: {distance_railway:.2f} km (transportation connectivity)

3. All launch sites are:
   - Close to equator for orbital mechanics advantages
   - Near coastlines for safety and trajectory clearance
   - Connected to transportation infrastructure
   - Appropriately distanced from populated areas

Geographic Advantages:
✓ Equatorial proximity reduces energy requirements
✓ Coastal location provides safety buffer
✓ Transportation access enables efficient logistics
✓ Remote enough to ensure public safety
""")

# Display the final interactive map
print("\nDisplaying final interactive map with all analysis layers...")
site_map



=== Analysis Complete ===
TASK 3 Complete: Distance analysis and proximity mapping finished

=== GEOGRAPHIC INSIGHTS SUMMARY ===

Key Findings:
1. Launch sites are strategically positioned near coastlines for safety and trajectory optimization
2. KSC LC-39A distances:
   - Coastline: 7.41 km (very close for safety)
   - Highway: 6.76 km (accessible for logistics)
   - City: 21.16 km (safe distance from population)
   - Railway: 7.77 km (transportation connectivity)

3. All launch sites are:
   - Close to equator for orbital mechanics advantages
   - Near coastlines for safety and trajectory clearance
   - Connected to transportation infrastructure
   - Appropriately distanced from populated areas

Geographic Advantages:
✓ Equatorial proximity reduces energy requirements
✓ Coastal location provides safety buffer
✓ Transportation access enables efficient logistics
✓ Remote enough to ensure public safety


Displaying final interactive map with all analysis layers...


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
